## Project: “The I in Team” – NFL Top Offensive Players & Team Success
Analyzing top offensive NFL players in each of their positions and seeing how their team did that season. Common sense will tell you that having a top QB would be more important to a team's success than 
a TE, but will the data support that. 

Summary:
    
    -Positions: QB, RB, WR, TE

    -Top players per season: Top 3 in each position

    -Timeframe: Last 13 NFL seasons (~2012–2024)

    -Metrics for ranking:
        -Total touchdowns (rushing + receiving)
        -Passing/Rushing/Recieving yards
            -Passing - QB
            -Rushing - RB
            -Receiving - WR, TE
        -Fantasy points (standard scoring)

    -Data Source: 
    -kaggle datasets:
        -"NFL Stats 2012-2024"
        -"1926-2024 NFL Scores"
    

# Imports and Setup
We import the necessary data and libraries for data analysis and visualization. 

In [35]:
import pandas as pd


# Data Collection

In [36]:
yearly_data = pd.read_csv("data/yearly_player_stats_offense.csv")
weekly_results = pd.read_csv("data/1926-2024_COMBINED_NFL_SCORES.csv")

Check the shape to get a general idea of the data.

In [37]:
yearly_data.shape

(7133, 660)

First we will separate out the data by position. Creating 4 dataframes for each respective position.



In [38]:
#create dataframe with only the QBs and exclude postseason
mask = (yearly_data["position"] == "QB") & (yearly_data["season_type"] == "REG")
QB_data_only = yearly_data[mask]
QB_data_organized = QB_data_only[["player_name", "team", "passing_yards", "pass_touchdown", "position", "season", "fantasy_points_standard"]].copy()
#display and check format of data
QB_data_organized.head()

,player_name,team,passing_yards,pass_touchdown,position,season,fantasy_points_standard
0,Charlie Batch,PIT,475.0,1.0,QB,2012,17.00
4,Matt Hasselbeck,TEN,1367.0,7.0,QB,2012,87.48
6,Peyton Manning,DEN,4659.0,37.0,QB,2012,380.96
12,Tom Brady,NE,4827.0,34.0,QB,2012,414.28
18,Drew Brees,NO,5177.0,43.0,QB,2012,430.58


In [39]:
unique_teams = QB_data_organized["team"].unique()
print(unique_teams)

['PIT' 'TEN' 'DEN' 'NE' 'NO' 'NYG' 'LV' 'HOU' 'LAC' 'SF' 'GB' 'CHI' 'DAL'
 'TB' 'CAR' 'KC' 'BUF' 'LA' 'CIN' 'ARI' 'ATL' 'BAL' 'JAX' 'SEA' 'DET'
 'NYJ' 'CLE' 'MIN' 'PHI' 'WAS' 'IND' 'MIA']


In [40]:
#create dataframe with only the RBs and exclude postseason
mask = (yearly_data["position"] == "RB") & (yearly_data["season_type"] == "REG")
RB_data_only = yearly_data[mask]
RB_data_organized = RB_data_only[["player_name", "team", "rushing_yards", "rush_touchdown", "position", "season", "fantasy_points_standard"]].copy()
#display and check format of data
RB_data_organized.head()

,player_name,team,rushing_yards,rush_touchdown,position,season,fantasy_points_standard
38,Willis McGahee,DEN,731.0,4.0,RB,2012,107.2
39,Mewelde Moore,IND,14.0,0.0,RB,2012,15.1
40,Steven Jackson,LA,1045.0,4.0,RB,2012,160.6
48,Michael Turner,ATL,800.0,10.0,RB,2012,155.8
55,Ronnie Brown,LAC,189.0,0.0,RB,2012,55.2


In [41]:
#create dataframe with only the TEs and exclude postseason
mask = (yearly_data["position"] == "TE") & (yearly_data["season_type"] == "REG")
TE_data_only = yearly_data[mask]
TE_data_organized = TE_data_only[["player_name",  "team", "receiving_yards", "receiving_touchdown", "position", "season", "fantasy_points_standard"]].copy()
#display and check format of data
TE_data_organized.head()

,player_name,team,receiving_yards,receiving_touchdown,position,season,fantasy_points_standard
3,Tony Gonzalez,ATL,930.0,8.0,TE,2012,135.0
17,Todd Heap,ARI,94.0,0.0,TE,2012,9.4
23,Randy McMichael,LAC,51.0,0.0,TE,2012,5.1
30,Visanthe Shiancoe,NE,0.0,0.0,TE,2012,-2.0
35,Jason Witten,DAL,1039.0,3.0,TE,2012,115.9


In [42]:
#create dataframe with only the WRs and exclude postseason
mask = (yearly_data["position"] == "WR") & (yearly_data["season_type"] == "REG")
WR_data_only = yearly_data[mask]
WR_data_organized = WR_data_only[["player_name", "team", "receiving_yards", "receiving_touchdown", "position", "season", "fantasy_points_standard"]].copy()
#display and check format of data
WR_data_organized.head()

,player_name,team,receiving_yards,receiving_touchdown,position,season,fantasy_points_standard
1,Donald Driver,GB,77.0,2.0,WR,2012,19.7
8,Randy Moss,SF,434.0,3.0,WR,2012,55.4
10,Brandon Stokley,DEN,544.0,5.0,WR,2012,80.4
13,Plaxico Burress,PIT,42.0,1.0,WR,2012,10.2
15,Santana Moss,WAS,573.0,8.0,WR,2012,101.7


Now that the data has been split into separate dataframes based on player position, we can write a formula to generate a dataframe for the top 3 of each position.

In [43]:
weekly_results.dtypes

Date       object
DOW        object
WT         object
LT         object
WTS       float64
LTS       float64
Type       object
Season      int64
dtype: object

We notice that the Season is an object. Converting it to an integer will allow us to compare values easily.

In [44]:

weekly_results["Season"] = weekly_results["Season"].astype(int)

In [45]:
#filter for playoff games
mask = (weekly_results["Type"] == "Playoff") & (weekly_results["Season"] > 2012)
weekly_results_playoffs_only = weekly_results[mask]
weekly_results_playoffs_only.head()

,Date,DOW,WT,LT,WTS,LTS,Type,Season
13674,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2013
13675,2014-01-04,Sat,Indianapolis Colts,Kansas City Chiefs,45.0,44.0,Playoff,2013
13676,2014-01-04,Sat,New Orleans Saints,Philadelphia Eagles,26.0,24.0,Playoff,2013
13677,2014-01-05,Sun,San Diego Chargers,Cincinnati Bengals,27.0,10.0,Playoff,2013
13678,2014-01-05,Sun,San Francisco 49ers,Green Bay Packers,23.0,20.0,Playoff,2013


We notice there may be some discrepancies. We want to see how many values are not available in our new data set. 

In [46]:
weekly_results_playoffs_only[weekly_results_playoffs_only["Date"].str.contains("Playoff", na=False)]

,Date,DOW,WT,LT,WTS,LTS,Type,Season
13674,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2013
13942,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2014
14210,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2015
14478,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2016
14746,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2017
15014,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2018
15282,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2019
15550,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2020
15836,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2021
16121,Playoffs,NaN,NaN,NaN,NaN,NaN,Playoff,2022


I think these were input to mark the change from regular season to playoffs in the original dataframe. But we can double check.
From 2013 - 2019, there were 11 games each postseason. From 2020-2024, there were 14 games. 
7 x 11 + 5 x 13 = 142

In [47]:
print(weekly_results_playoffs_only['Season'].value_counts())

Season
2020    14
2021    14
2022    14
2023    14
2024    14
2013    12
2014    12
2015    12
2016    12
2017    12
2018    12
2019    12
Name: count, dtype: int64


In [48]:
weekly_results_playoffs_only.shape

(154, 8)

It looks like the difference between the columns and the number of playoff games there should be is 12 (the number of seasons), which further reinforces the idea that the rows with "Playoffs" were just placeholders carried over from the previous dataframe. 
We can double-check though, by inspecting a cross-section of the data against what we can find online. 

In [49]:
filtered_df_2014 = weekly_results_playoffs_only.loc[weekly_results_playoffs_only["Season"] == 2014]
filtered_df_2024 = weekly_results_playoffs_only.loc[weekly_results_playoffs_only["Season"] == 2024]

In [50]:
print(filtered_df_2014)

             Date  DOW                    WT                   LT   WTS   LTS  \
13942    Playoffs  NaN                   NaN                  NaN   NaN   NaN   
13943  2015-01-03  Sat     Carolina Panthers    Arizona Cardinals  27.0  16.0   
13944  2015-01-03  Sat      Baltimore Ravens  Pittsburgh Steelers  30.0  17.0   
13945  2015-01-04  Sun    Indianapolis Colts   Cincinnati Bengals  26.0  10.0   
13946  2015-01-04  Sun        Dallas Cowboys        Detroit Lions  24.0  20.0   
13947  2015-01-10  Sat  New England Patriots     Baltimore Ravens  35.0  31.0   
13948  2015-01-10  Sat      Seattle Seahawks    Carolina Panthers  31.0  17.0   
13949  2015-01-11  Sun     Green Bay Packers       Dallas Cowboys  26.0  21.0   
13950  2015-01-11  Sun    Indianapolis Colts       Denver Broncos  24.0  13.0   
13951  2015-01-18  Sun      Seattle Seahawks    Green Bay Packers  28.0  22.0   
13952  2015-01-18  Sun  New England Patriots   Indianapolis Colts  45.0   7.0   
13953  2015-02-01  Sun  New 

In [51]:
print(filtered_df_2024)

             Date  DOW                     WT                     LT   WTS  \
16693    Playoffs  NaN                    NaN                    NaN   NaN   
16694  2025-01-11  Sat         Houston Texans   Los Angeles Chargers  32.0   
16695  2025-01-11  Sat       Baltimore Ravens    Pittsburgh Steelers  28.0   
16696  2025-01-12  Sun          Buffalo Bills         Denver Broncos  31.0   
16697  2025-01-12  Sun    Philadelphia Eagles      Green Bay Packers  22.0   
16698  2025-01-12  Sun  Washington Commanders   Tampa Bay Buccaneers  23.0   
16699  2025-01-13  Mon       Los Angeles Rams      Minnesota Vikings  27.0   
16700  2025-01-18  Sat     Kansas City Chiefs         Houston Texans  23.0   
16701  2025-01-18  Sat  Washington Commanders          Detroit Lions  45.0   
16702  2025-01-19  Sun    Philadelphia Eagles       Los Angeles Rams  28.0   
16703  2025-01-19  Sun          Buffalo Bills       Baltimore Ravens  27.0   
16704  2025-01-26  Sun    Philadelphia Eagles  Washington Comman

Now that we have verified multiple ways. Lets remove the columns where N/As are present since it seems like that will coincide with those with "Playoff" in the Date column.

In [52]:
weekly_results_playoffs_only = weekly_results_playoffs_only.dropna()
#check to see the shape, now expecting 142 rows
weekly_results_playoffs_only.shape

(142, 8)

In [53]:
weekly_results_playoffs_only.head(12)

,Date,DOW,WT,LT,WTS,LTS,Type,Season
13675,2014-01-04,Sat,Indianapolis Colts,Kansas City Chiefs,45.0,44.0,Playoff,2013
13676,2014-01-04,Sat,New Orleans Saints,Philadelphia Eagles,26.0,24.0,Playoff,2013
13677,2014-01-05,Sun,San Diego Chargers,Cincinnati Bengals,27.0,10.0,Playoff,2013
13678,2014-01-05,Sun,San Francisco 49ers,Green Bay Packers,23.0,20.0,Playoff,2013
13679,2014-01-11,Sat,Seattle Seahawks,New Orleans Saints,23.0,15.0,Playoff,2013
13680,2014-01-11,Sat,New England Patriots,Indianapolis Colts,43.0,22.0,Playoff,2013
13681,2014-01-12,Sun,San Francisco 49ers,Carolina Panthers,23.0,10.0,Playoff,2013
13682,2014-01-12,Sun,Denver Broncos,San Diego Chargers,24.0,17.0,Playoff,2013
13683,2014-01-19,Sun,Denver Broncos,New England Patriots,26.0,16.0,Playoff,2013
13684,2014-01-19,Sun,Seattle Seahawks,San Francisco 49ers,23.0,17.0,Playoff,2013


In [54]:
def encode_playoff_rounds(df):
    """
    Returns a pd.Series of encoded round numbers for a DataFrame that includes only playoff games.
    1=Wild Card, 2=Divisional, 3=Conference, 4=Super Bowl
    Handles pre/post 2020 playoff formats.
    """
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Initialize Series
    round_codes = pd.Series(index=df.index, dtype=int)
    
    # Process season by season
    for season, group in df.groupby("Season"):
        group = group.sort_values("Date")
        n_games = len(group)
        
        # Determine breaks
        if season <= 2019:
            # 11 games: 4 WC, 4 Div, 2 Conf, 1 SB
            round_breaks = [4, 8, 10, 11]
        else:
            # 14 games: 6 WC, 4 Div, 2 Conf, 1 SB, 1 optional winner?
            round_breaks = [6, 10, 12, 13]
        
        codes = []
        for i in range(n_games):
            if i < round_breaks[0]:
                codes.append(1)
            elif i < round_breaks[1]:
                codes.append(2)
            elif i < round_breaks[2]:
                codes.append(3)
            elif i < round_breaks[3]:
                codes.append(4)
        
        round_codes.loc[group.index] = codes
    
    return round_codes

In [55]:
weekly_results_playoffs_only["encoded_number"] = encode_playoff_rounds(weekly_results_playoffs_only)
weekly_results_playoffs_only.head(10)

,Date,DOW,WT,LT,WTS,LTS,Type,Season,encoded_number
13675,2014-01-04,Sat,Indianapolis Colts,Kansas City Chiefs,45.0,44.0,Playoff,2013,1.0
13676,2014-01-04,Sat,New Orleans Saints,Philadelphia Eagles,26.0,24.0,Playoff,2013,1.0
13677,2014-01-05,Sun,San Diego Chargers,Cincinnati Bengals,27.0,10.0,Playoff,2013,1.0
13678,2014-01-05,Sun,San Francisco 49ers,Green Bay Packers,23.0,20.0,Playoff,2013,1.0
13679,2014-01-11,Sat,Seattle Seahawks,New Orleans Saints,23.0,15.0,Playoff,2013,2.0
13680,2014-01-11,Sat,New England Patriots,Indianapolis Colts,43.0,22.0,Playoff,2013,2.0
13681,2014-01-12,Sun,San Francisco 49ers,Carolina Panthers,23.0,10.0,Playoff,2013,2.0
13682,2014-01-12,Sun,Denver Broncos,San Diego Chargers,24.0,17.0,Playoff,2013,2.0
13683,2014-01-19,Sun,Denver Broncos,New England Patriots,26.0,16.0,Playoff,2013,3.0
13684,2014-01-19,Sun,Seattle Seahawks,San Francisco 49ers,23.0,17.0,Playoff,2013,3.0


In [56]:
season_results = weekly_results_playoffs_only[["Season", "WT", "LT", "encoded_number"]]
# Create two DataFrames: winners and losers
winners = season_results[["Season", "WT", "encoded_number"]].rename(columns={"WT": "Team"})
losers  = season_results[["Season", "LT", "encoded_number"]].rename(columns={"LT": "Team"})

# Concatenate to have one row per team per game
all_teams = pd.concat([winners, losers], ignore_index=True)
team_season_summary = all_teams.groupby(["Season", "Team"])["encoded_number"].max().reset_index()
# Filter Super Bowl games (encoded_number == 4)
superbowls = weekly_results_playoffs_only[weekly_results_playoffs_only["encoded_number"] == 4]

# List of (season, team) that won the Super Bowl
sb_winners = superbowls[["Season", "WT"]].rename(columns={"WT": "Team"})
sb_winners["encoded_number"] = 5 
team_season_summary = team_season_summary.merge(sb_winners, on=["Season","Team"], how="left", suffixes=("", "_SB"))

# If SB winner, use 5, otherwise keep original
team_season_summary["encoded_number"] = team_season_summary["encoded_number_SB"].combine_first(team_season_summary["encoded_number"])

# Drop helper column
team_season_summary = team_season_summary.drop(columns=["encoded_number_SB"])

team_season_summary.head(12)

,Season,Team,encoded_number
0,2013,Carolina Panthers,2.0
1,2013,Cincinnati Bengals,1.0
2,2013,Denver Broncos,4.0
3,2013,Green Bay Packers,1.0
4,2013,Indianapolis Colts,2.0
5,2013,Kansas City Chiefs,1.0
6,2013,New England Patriots,3.0
7,2013,New Orleans Saints,2.0
8,2013,Philadelphia Eagles,1.0
9,2013,San Diego Chargers,2.0


Lets check that everthing is expected before transforming the data in the summary table.

In [57]:
summary_table = (
    team_season_summary
    .groupby("Season")["encoded_number"]
    .value_counts()
    .unstack(fill_value=0)  # put encoded numbers as columns
    .sort_index(axis=1)     # optional: order columns 1-5
)
summary_table = summary_table.rename(columns={
    1: "WildCard",
    2: "Divisional",
    3: "Conference",
    4: "SuperBowlLoser",
    5: "SuperBowlWinner"
})
summary_table["Total"] = summary_table.sum(axis=1)

summary_table

encoded_number,WildCard,Divisional,Conference,SuperBowlLoser,SuperBowlWinner,Total
Season,,,,,,
2013,4,4,2,1,1,12
2014,4,4,2,1,1,12
2015,4,4,2,1,1,12
2016,4,4,2,1,1,12
2017,4,4,2,1,1,12
2018,4,4,2,1,1,12
2019,4,4,2,1,1,12
2020,6,4,2,1,1,14
2021,6,4,2,1,1,14


Now we can make the names of the teams match. (i.e. Atlanta Falcons to ATL)

In [58]:
team_season_summary["Team"].unique()

array(['Carolina Panthers', 'Cincinnati Bengals', 'Denver Broncos',
       'Green Bay Packers', 'Indianapolis Colts', 'Kansas City Chiefs',
       'New England Patriots', 'New Orleans Saints',
       'Philadelphia Eagles', 'San Diego Chargers', 'San Francisco 49ers',
       'Seattle Seahawks', 'Arizona Cardinals', 'Baltimore Ravens',
       'Dallas Cowboys', 'Detroit Lions', 'Pittsburgh Steelers',
       'Houston Texans', 'Minnesota Vikings', 'Washington Redskins',
       'Atlanta Falcons', 'Miami Dolphins', 'New York Giants',
       'Oakland Raiders', 'Buffalo Bills', 'Jacksonville Jaguars',
       'Los Angeles Rams', 'Tennessee Titans', 'Chicago Bears',
       'Los Angeles Chargers', 'Cleveland Browns', 'Tampa Bay Buccaneers',
       'Washington Football Team', 'Las Vegas Raiders',
       'Washington Commanders'], dtype=object)

Generated the library below with Chatgpt. 

In [59]:
team_abbrev_map = {
    # AFC East
    "Buffalo Bills": "BUF",
    "Miami Dolphins": "MIA",
    "New England Patriots": "NE",
    "New York Jets": "NYJ",
    
    # AFC North
    "Baltimore Ravens": "BAL",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Pittsburgh Steelers": "PIT",
    
    # AFC South
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Tennessee Titans": "TEN",
    
    # AFC West
    "Denver Broncos": "DEN",
    "Kansas City Chiefs": "KC",
    "Oakland Raiders": "LV",
    "Las Vegas Raiders": "LV",
    "San Diego Chargers": "LAC",
    "Los Angeles Chargers": "LAC",
    
    # NFC East
    "Dallas Cowboys": "DAL",
    "New York Giants": "NYG",
    "Philadelphia Eagles": "PHI",
    "Washington Redskins": "WAS",
    "Washington Football Team": "WAS",
    "Washington Commanders": "WAS",
    
    # NFC North
    "Chicago Bears": "CHI",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Minnesota Vikings": "MIN",
    
    # NFC South
    "Atlanta Falcons": "ATL",
    "Carolina Panthers": "CAR",
    "New Orleans Saints": "NO",
    "Tampa Bay Buccaneers": "TB",
    
    # NFC West
    "Arizona Cardinals": "ARI",
    "Los Angeles Rams": "LA",
    "St. Louis Rams": "LA",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA"
}

In [60]:
team_season_summary["Team"] = team_season_summary["Team"].replace(team_abbrev_map)
team_season_summary.head(10)

,Season,Team,encoded_number
0,2013,CAR,2.0
1,2013,CIN,1.0
2,2013,DEN,4.0
3,2013,GB,1.0
4,2013,IND,2.0
5,2013,KC,1.0
6,2013,NE,3.0
7,2013,NO,2.0
8,2013,PHI,1.0
9,2013,LAC,2.0


In [61]:
# Teams that made playoffs
playoff_teams = team_season_summary["Team"].unique()

# Teams represented in your QB dataset
QB_teams = QB_data_organized["team"].unique()
missing_teams = set(playoff_teams) - set(QB_teams)
print("Playoff teams not in QB dataset:", missing_teams)

Playoff teams not in QB dataset: set()


Now that we have the names matched between the datasets. We can begin tying the top performing players to their teams result.

In [62]:

merged_QB_df = QB_data_organized.merge(
    team_season_summary[["Season", "Team", "encoded_number"]],
    left_on=["season", "team"],   # lowercase 'team' from QB_data_organized
    right_on=["Season", "Team"],  # 'Team' from summary table
    how="left"
)
merged_RB_df = RB_data_organized.merge(
    team_season_summary[["Season", "Team", "encoded_number"]],
    left_on=["season", "team"],   # lowercase 'team' from QB_data_organized
    right_on=["Season", "Team"],  # 'Team' from summary table
    how="left"
)
merged_WR_df = WR_data_organized.merge(
    team_season_summary[["Season", "Team", "encoded_number"]],
    left_on=["season", "team"],   # lowercase 'team' from QB_data_organized
    right_on=["Season", "Team"],  # 'Team' from summary table
    how="left"
)
merged_TE_df = TE_data_organized.merge(
    team_season_summary[["Season", "Team", "encoded_number"]],
    left_on=["season", "team"],   # lowercase 'team' from QB_data_organized
    right_on=["Season", "Team"],  # 'Team' from summary table
    how="left"
)

Then we can make a list of dataframes holding each seasons top 3 in each category.

We can write a function to create a dataframe with the top 3 of a position based on some statistic.

In [63]:
# Function will take a dataframe, the season, the stat being investigated(stat_col), and the cutoff for top players (defaulted to 3)
def top_players_by_season(df, season, stat_col, encoded_number, top_n=3):
    """
    Return the top N players for a given season and stat column.
    """
    season_data = df[df["season"] == season]
    return (season_data
            .sort_values(by=stat_col, ascending=False)
            .head(top_n)[["player_name", stat_col, "team", "season", "encoded_number"]])

# Example: Top 3 RBs by rushing yards in 2019
top_players_by_season(merged_QB_df, 2020, "passing_yards", "encoded_number",)

,player_name,passing_yards,team,season,encoded_number
532,Deshaun Watson,4823.0,HOU,2020,NaN
534,Patrick Mahomes,4740.0,KC,2020,4.0
489,Tom Brady,4633.0,TB,2020,5.0


A list of dataframes representing the top 3 of each season is generated for each position and required statistic.

In [64]:
top3_list_QB_pass_yds = []
for season in merged_QB_df['season'].unique():
    top3_season_yds = top_players_by_season(merged_QB_df, season, "passing_yards", "encoded_number")
    top3_list_QB_pass_yds.append(top3_season_yds)

top3_list_QB_pass_tds = []
for season in merged_QB_df['season'].unique():
    top3_season_tds = top_players_by_season(merged_QB_df, season, "pass_touchdown", "encoded_number")
    top3_list_QB_pass_tds.append(top3_season_tds)

top3_list_QB_fantasy = []
for season in merged_QB_df['season'].unique():
    top3_season_fant = top_players_by_season(merged_QB_df, season, "fantasy_points_standard", "encoded_number")
    top3_list_QB_fantasy.append(top3_season_fant)

In [ ]:
top3_list_RB_rush_yds = []
for season in merged_RB_df['season'].unique():
    top3_season_yds = top_players_by_season(merged_RB_df, season, "rushing_yards", "encoded_number")
    top3_list_RB_rush_yds.append(top3_season_yds)

top3_list_RB_rush_tds = []
for season in merged_RB_df['season'].unique():
    top3_season_tds = top_players_by_season(merged_RB_df, season, "rush_touchdown", "encoded_number")
    top3_list_RB_rush_tds.append(top3_season_tds)

top3_list_RB_fantasy = []
for season in merged_RB_df['season'].unique():
    top3_season_fant = top_players_by_season(merged_RB_df, season, "fantasy_points_standard", "encoded_number")
    top3_list_RB_fantasy.append(top3_season_fant)



In [ ]:
top3_list_WR_rec_yds = []
for season in merged_WR_df['season'].unique():
    top3_season_yds = top_players_by_season(merged_WR_df, season, "receiving_yards", "encoded_number")
    top3_list_WR_rec_yds.append(top3_season_yds)

top3_list_WR_rec_tds = []
for season in merged_WR_df['season'].unique():
    top3_season_tds = top_players_by_season(merged_WR_df, season, "receiving_touchdown", "encoded_number")
    top3_list_WR_rec_tds.append(top3_season_tds)

top3_list_WR_fantasy = []
for season in merged_WR_df['season'].unique():
    top3_season_fant = top_players_by_season(merged_WR_df, season, "fantasy_points_standard", "encoded_number")
    top3_list_WR_fantasy.append(top3_season_fant)


In [ ]:
top3_list_TE_rec_yds = []
for season in merged_TE_df['season'].unique():
    top3_season_yds = top_players_by_season(merged_TE_df, season, "receiving_yards", "encoded_number")
    top3_list_TE_rec_yds.append(top3_season_yds)

top3_list_TE_rec_tds = []
for season in merged_TE_df['season'].unique():
    top3_season_tds = top_players_by_season(merged_TE_df, season, "receiving_touchdown", "encoded_number")
    top3_list_TE_rec_tds.append(top3_season_tds)

top3_list_TE_fantasy = []
for season in merged_TE_df['season'].unique():
    top3_season_fant = top_players_by_season(merged_TE_df, season, "fantasy_points_standard", "encoded_number")
    top3_list_TE_fantasy.append(top3_season_fant)
